In [1]:
from __future__ import unicode_literals
import classifiers
import re
import nltk
from Politweet import get_tweets, get_transcript
import ratings
from sentiment import polarity_train, classify, prob_classify, plus_df, minus_df
import pandas as pd
pd.set_option('display.max_colwidth', 1200)

In [2]:
tweets = get_tweets("./datasets/tweets.tsv")
transcript = get_transcript("./datasets/transcript.csv")
todo = set(tweets.index)

In [3]:
polarity = polarity_train(tweets)
polarity_minus = set(i for i,_ in classify(minus_df(tweets), polarity))
polarity_plus = set(i for i,_ in classify(plus_df(tweets), polarity))
polarities = tweets.reindex(polarity_minus | polarity_plus).sort()
print len(polarities), "- polarity"
todo -= set(polarities.index)

482 - polarity


/Library/Python/2.7/site-packages/pandas/core/strings.py:189: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  " groups, use str.extract.", UserWarning)


In [4]:
others = ratings.all(tweets, ratings.OTHER)
print len(others), "- marked as OTHERS"
todo -= set(others.index)

151 - marked as OTHERS


In [5]:
print len(todo), "- untagged"

2605 - untagged


In [6]:
left = tweets.reindex(todo).sort()
left[:1]

,pub.date.GMT,content,author.name,author.nickname,rating.1,rating.2,rating.3,rating.4,rating.5,rating.6,rating.7,rating.8,tokens,clean
tweet.id,,,,,,,,,,,,,,
936466790,9/27/08 1:00,Jim Lehrer just directed the debate audience ... 30 seconds ... #tweetdebate,B_E_X,Becky L,4,2,4,NaN,NaN,NaN,NaN,NaN,"[Jim, Lehrer, just, directed, the, debate, audience, ..., 30, seconds, ..., #, tweetdebate]","[{u'lemma': u'Jim', u'token': u'Jim', u'pos': u'NNP'}, {u'lemma': u'Lehrer', u'token': u'Lehrer', u'pos': u'NNP'}, {u'lemma': u'direct', u'token': u'directed', u'pos': u'VBD'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'audience', u'token': u'audience', u'pos': u'NN'}, {u'lemma': u'...', u'token': u'...', u'pos': u':'}, {u'lemma': u'30', u'token': u'30', u'pos': u'CD'}, {u'lemma': u'second', u'token': u'seconds', u'pos': u'NNS'}, {u'lemma': u'...', u'token': u'...', u'pos': u':'}, {u'lemma': u'#tweetdebate', u'token': u'#tweetdebate', u'pos': u'NN'}]"


In [16]:
from topic_extraction import remove_stopwords
others[["tokens"]]

,tokens
tweet.id,
936468396,"[@, current, one, minute, to, get, your, stuff, together, #, current]"
936468782,"[Anyone, else, actually, kinda, nervous, ?, #, tweetdebate]"
936470593,"[#, tweetdebate, ready, ,, set, ,, go, !]"
936471247,"[Any, drinking, game, ideas, for, #, tweetdebate, ?]"
936473721,"[@, mojosd, refresh, ?]"
936475560,"[#, tweetdebate, i, never, knew, what, the, symbol, of, the, Dems, was, till, 12, yrs, ago]"
936475824,"[@, starweaver, what, happened, ?]"
936480186,"[#, tweetdebate]"
936485053,"[Just, started, watching, the, debate, ., Got, home, a, few, mins, late, ., Got, to, get, #, current, and, then, #, tweetdebate]"


In [14]:
for i, tweet in tweets[:3].iterrows():
    print [token["lemma"] for token in tweet["clean"]]

[u'Watching', u'#', u'tweetdebate', u'drink', u':', u'(', u'wait', u'start', u'cringing', u'McCain', u'blunder']
[u'@', u'ahg3', u'@', u'MichDot', u'Yeah', u',', u'slime', u'actually', u'second', u'choice', u',', u'ca', u"n't", u'say', u'first', u'one', u'.', u'Okay', u',', u"'re", u'roll', u'...']
[u'Preparing', u'heart', u'attack', u'#', u'tweetdebate']
